<a href="https://colab.research.google.com/github/KCW-MIMI/Final_exam/blob/main/2018802037_%EA%B9%80%EC%B0%AC%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForSequenceClassification, pipeline
from datasets import load_dataset
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import evaluate

# 문제 1

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
! pip install transformers datasets evaluate

In [19]:
from datasets import load_dataset

potter = load_dataset('KaungHtetCho/Harry_Potter_LSTM')

None of them noticed a large, tawny owl flutter past the window.

In [5]:
# for reproducibility
torch.manual_seed(100)

# Data Preparation
sample_sentence = "None of them noticed a large, tawny owl flutter past the window."
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# Dataset setting
input_batch = []
target_batch = []
for i in range(len(sample_sentence) - unit_sequence_length):
    x_data = [dic[c] for c in sample_sentence[i:i + unit_sequence_length]]
    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    y_data = [dic[c] for c in sample_sentence[i + 1:i + unit_sequence_length + 1]]
    input_batch.append(x_one_hot)
    target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))

# Model Definition
class Custom_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Custom_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

# Training Parameters
learning_rate = 0.05
training_epochs = 100
model = Custom_LSTM(input_size, hidden_size, output_size, 2)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train
for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch % 10 == 9:
        print(f'epoch: {epoch}, loss: {loss.item()}')

# Predict and Evaluate
model.eval()
with torch.no_grad():
    predicted_sequence = ""
    current_sequence = sample_sentence[:unit_sequence_length]
    for _ in range(len(sample_sentence) - unit_sequence_length):
        x_data = [dic[c] for c in current_sequence]
        x_one_hot = [np.eye(dic_size)[x] for x in x_data]
        X_pred = torch.FloatTensor(np.array([x_one_hot]))

        output = model(X_pred)
        predicted_idx = output.argmax(dim=2)[0, -1].item()
        predicted_char = char_set[predicted_idx]
        predicted_sequence += predicted_char

        current_sequence = current_sequence[1:] + predicted_char

    target = sample_sentence[unit_sequence_length:]
    correct = sum(1 for x, y in zip(predicted_sequence, target) if x == y)
    accuracy = correct / len(target)

    print(f"\nPredicted sequence: {predicted_sequence[:100]}...")
    print(f"Original sequence: {target[:100]}...")
    print(f"Accuracy: {accuracy:.4f}")

epoch: 9, loss: 2.576260566711426
epoch: 19, loss: 1.1021710634231567
epoch: 29, loss: 0.2477666288614273
epoch: 39, loss: 0.09566905349493027
epoch: 49, loss: 0.06606940180063248
epoch: 59, loss: 0.05809406936168671
epoch: 69, loss: 0.05543052777647972
epoch: 79, loss: 0.05440155416727066
epoch: 89, loss: 0.05389054864645004
epoch: 99, loss: 0.05359654128551483

Predicted sequence:  a large, tawny owl flutter past the window....
Original sequence:  a large, tawny owl flutter past the window....
Accuracy: 1.0000


In [6]:
# for reproducibility
torch.manual_seed(100)

# Data Preparation
sample_sentence = "None of them noticed a large, tawny owl flutter past the window."
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# Dataset setting
input_batch = []
target_batch = []
for i in range(len(sample_sentence) - unit_sequence_length):
    x_data = [dic[c] for c in sample_sentence[i:i + unit_sequence_length]]
    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    y_data = [dic[c] for c in sample_sentence[i + 1:i + unit_sequence_length + 1]]
    input_batch.append(x_one_hot)
    target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))

# Model Definition
class Custom_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Custom_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

# Training Parameters
learning_rate = 0.05
training_epochs = 100
model = Custom_LSTM(input_size, hidden_size, output_size, 2)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train
for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch % 10 == 9:
        print(f'epoch: {epoch}, loss: {loss.item()}')

# Predict and Evaluate
model.eval()
with torch.no_grad():
    predicted_sequence = ""
    current_sequence = sample_sentence[:unit_sequence_length]
    for _ in range(len(sample_sentence) - unit_sequence_length):
        x_data = [dic[c] for c in current_sequence]
        x_one_hot = [np.eye(dic_size)[x] for x in x_data]
        X_pred = torch.FloatTensor(np.array([x_one_hot]))

        output = model(X_pred)
        predicted_idx = output.argmax(dim=2)[0, -1].item()
        predicted_char = char_set[predicted_idx]
        predicted_sequence += predicted_char

        current_sequence = current_sequence[1:] + predicted_char

    target = sample_sentence[unit_sequence_length:]
    correct = sum(1 for x, y in zip(predicted_sequence, target) if x == y)
    accuracy = correct / len(target)

    print(f"\nPredicted sequence: {predicted_sequence[:100]}...")
    print(f"Original sequence: {target[:100]}...")
    print(f"Accuracy: {accuracy:.4f}")

epoch: 9, loss: 2.576260566711426
epoch: 19, loss: 1.1021710634231567
epoch: 29, loss: 0.2477666288614273
epoch: 39, loss: 0.09566905349493027
epoch: 49, loss: 0.06606940180063248
epoch: 59, loss: 0.05809406936168671
epoch: 69, loss: 0.05543052777647972
epoch: 79, loss: 0.05440155416727066
epoch: 89, loss: 0.05389054864645004
epoch: 99, loss: 0.05359654128551483

Predicted sequence:  a large, tawny owl flutter past the window....
Original sequence:  a large, tawny owl flutter past the window....
Accuracy: 1.0000


In [7]:
# for reproducibility
torch.manual_seed(100)

# Data Preparation
sample_sentence = "None of them noticed a large, tawny owl flutter past the window."
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# Dataset setting
input_batch = []
target_batch = []
for i in range(len(sample_sentence) - unit_sequence_length):
    x_data = [dic[c] for c in sample_sentence[i:i + unit_sequence_length]]
    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    y_data = [dic[c] for c in sample_sentence[i + 1:i + unit_sequence_length + 1]]
    input_batch.append(x_one_hot)
    target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))

# Model Definition
class Custom_GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers):
        super(Custom_GRU, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.fc(x)
        return x

# Training Parameters
learning_rate = 0.05
training_epochs = 100
model = Custom_GRU(input_size, hidden_size, output_size, 2)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train
for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch % 10 == 9:
        print(f'epoch: {epoch}, loss: {loss.item()}')

# Predict and Evaluate
model.eval()
with torch.no_grad():
    predicted_sequence = ""
    current_sequence = sample_sentence[:unit_sequence_length]
    for _ in range(len(sample_sentence) - unit_sequence_length):
        x_data = [dic[c] for c in current_sequence]
        x_one_hot = [np.eye(dic_size)[x] for x in x_data]
        X_pred = torch.FloatTensor(np.array([x_one_hot]))

        output = model(X_pred)
        predicted_idx = output.argmax(dim=2)[0, -1].item()
        predicted_char = char_set[predicted_idx]
        predicted_sequence += predicted_char

        current_sequence = current_sequence[1:] + predicted_char

    target = sample_sentence[unit_sequence_length:]
    correct = sum(1 for x, y in zip(predicted_sequence, target) if x == y)
    accuracy = correct / len(target)

    print(f"\nPredicted sequence: {predicted_sequence[:100]}...")
    print(f"Original sequence: {target[:100]}...")
    print(f"Accuracy: {accuracy:.4f}")

epoch: 9, loss: 1.9289137125015259
epoch: 19, loss: 0.28682053089141846
epoch: 29, loss: 0.09319930523633957
epoch: 39, loss: 0.061445679515600204
epoch: 49, loss: 0.055504873394966125
epoch: 59, loss: 0.0540645606815815
epoch: 69, loss: 0.053429361432790756
epoch: 79, loss: 0.053171269595623016
epoch: 89, loss: 0.053021740168333054
epoch: 99, loss: 0.05292179435491562

Predicted sequence:  a large, tawny owl flutter past the window....
Original sequence:  a large, tawny owl flutter past the window....
Accuracy: 1.0000


# 문제 2

In [8]:
# 사용할 모델 리스트
model_names = [
    'tabularisai/multilingual-sentiment-analysis',
    'Copycats/koelectra-base-v3-generalized-sentiment-analysis',
    'WhitePeak/bert-base-cased-Korean-sentiment'
]

# 각 모델에 대해 성능 평가
for model_name in model_names:
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    classifier = pipeline('sentiment-analysis', model=model_name, device=-1)

    # 예시 텍스트
    text = "이 영화는 정말 훌륭했습니다."
    prediction = classifier(text)
    print(f"모델: {model_name}, 예측: {prediction}")

모델: tabularisai/multilingual-sentiment-analysis, 예측: [{'label': 'LABEL_4', 'score': 0.7552888989448547}]
모델: Copycats/koelectra-base-v3-generalized-sentiment-analysis, 예측: [{'label': '1', 'score': 0.9824778437614441}]
모델: WhitePeak/bert-base-cased-Korean-sentiment, 예측: [{'label': 'LABEL_1', 'score': 0.9507705569267273}]


In [10]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
for i in range(3):
  # 한글 데이터셋 로드
  korean_dataset = load_dataset("sepidmnorozy/Korean_sentiment")
  # 테스트 데이터 샘플링
  test_sample_size = 100
  test_list = korean_dataset['test']
  rng = np.random.default_rng()
  random_indices = rng.choice(len(test_list), size=test_sample_size, replace=False)
  sampled_data = test_list.select(random_indices)



  models = ['tabularisai/multilingual-sentiment-analysis',
            'Copycats/koelectra-base-v3-generalized-sentiment-analysis',
            'WhitePeak/bert-base-cased-Korean-sentiment']

  for korean_model_name in models:
    korean_classifier = pipeline('sentiment-analysis', model=korean_model_name, device=-1)

    test_predictions = classifier(sampled_data['text'], max_length=512, truncation=True)
    predicted_labels = [label['label'] for label in test_predictions]
    mapped_labels = [model.config.label2id[label] for label in predicted_labels]
    reference_labels = sampled_data['label']
    print(korean_model_name)
    print(accuracy.compute(predictions=mapped_labels, references=reference_labels))
    print(reference_labels)
    print(mapped_labels)

tabularisai/multilingual-sentiment-analysis
{'accuracy': 0.75}
[1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0]
Copycats/koelectra-base-v3-generalized-sentiment-analysis
{'accuracy': 0.75}
[1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,

모든 모델이 점수가 같았다...